In [1]:
import pandas as pd
import os
import json
import re
import ast


### 1. Preprocessing

In [2]:
es_data = pd.DataFrame()
en_data = pd.DataFrame()

for part in range(21, 36):
    folder = f"part_{part}/"
    files = os.listdir(folder)
    
    for file in files:
        if file.endswith(".csv"):  
            filepath = os.path.join(folder, file)
            data_part = pd.read_csv(filepath, low_memory=False)
            
            es_part = data_part[data_part['lang'] == 'es']
            es_size = es_part.shape[0]
            
            if es_size > 0:
                en_part = data_part[data_part['lang'] == 'en'].sample(es_size, random_state=442)
                
                es_data = pd.concat([es_data, es_part], axis=0) 
                en_data = pd.concat([en_data, en_part], axis=0)

es_data.reset_index(drop=True, inplace=True)
en_data.reset_index(drop=True, inplace=True)

/tmp/ipykernel_3099892/3284871486.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  es_data = pd.concat([es_data, es_part], axis=0)
/tmp/ipykernel_3099892/3284871486.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  en_data = pd.concat([en_data, en_part], axis=0)
/tmp/ipykernel_3099892/3284871486.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining 

In [3]:
print("English tweets raw count: ", en_data.shape[0], "\nSpanish tweets raw count: ", es_data.shape[0])

English tweets raw count:  215882 
Spanish tweets raw count:  215882


In [12]:
en_data.retweetedUserID.isna().sum()

370443

In [13]:
es_data.retweetedUserID.isna().sum()

371113

In [4]:
es_data.columns

Index(['id', 'text', 'url', 'epoch', 'media', 'retweetedTweet',
       'retweetedTweetID', 'retweetedUserID', 'id_str', 'lang', 'rawContent',
       'replyCount', 'retweetCount', 'likeCount', 'quoteCount',
       'conversationId', 'conversationIdStr', 'hashtags', 'mentionedUsers',
       'links', 'viewCount', 'quotedTweet', 'in_reply_to_screen_name',
       'in_reply_to_status_id_str', 'in_reply_to_user_id_str', 'location',
       'cash_app_handle', 'user', 'date', '_type', 'type', 'username', '0'],
      dtype='object')

In [5]:
# Keep relevant and non-empty columns
columns_to_keep = [
    'id', 'text', 'lang', 'epoch', 'hashtags', 'links', 
    'replyCount', 'retweetCount', 'likeCount', 'quoteCount', 
    'conversationId', 'mentionedUsers', 'user'
]

In [6]:
en_data = en_data[columns_to_keep]
es_data = es_data[columns_to_keep]


In [7]:
def user_to_dict(user):
    try:
        # User metadata interested in
        user_metadata = ['id_str', 'followersCount', 'friendsCount', 'statusesCount']
        
        metadata_dict = {}
        
        for metadata in user_metadata:
            pattern = rf"'{metadata}':\s*('[^']*'|\d+|True|False|None)"
            match = re.search(pattern, user)
            
            if match:
                metadata_dict[metadata] = match.group(1)  # Extract the matched value
            else:
                metadata_dict[metadata] = None
        
        return metadata_dict
    
    except Exception as e:
        print("Error occurred:", e)
        print("User data:", user)
        raise

In [8]:
en_data['user'] = en_data['user'].apply(user_to_dict)
es_data['user'] = es_data['user'].apply(user_to_dict)


In [9]:
# Flatten user metadata into separate columns
user_metadata = ['id_str', 'followersCount', 'friendsCount', 'statusesCount']
for meta in user_metadata:
    en_data[meta] = en_data['user'].apply(lambda x: x.get(meta))
    es_data[meta] = es_data['user'].apply(lambda x: x.get(meta))



In [10]:
en_data.drop(columns=['user'], inplace=True)
es_data.drop(columns=['user'], inplace=True)


In [11]:
def convert_format(field):
    return ast.literal_eval(field)

# Convert string representation of lists into lists
en_data['hashtags'] = en_data['hashtags'].apply(convert_format)
en_data['links'] = en_data['links'].apply(convert_format)
en_data['mentionedUsers'] = en_data['mentionedUsers'].apply(convert_format)

es_data['hashtags'] = es_data['hashtags'].apply(convert_format)
es_data['links'] = es_data['links'].apply(convert_format)
es_data['mentionedUsers'] = es_data['mentionedUsers'].apply(convert_format)


In [13]:
en = pd.read_csv("en.csv")
es = pd.read_csv("es.csv")


In [29]:
# en['hashtags'] = en['hashtags'].apply(convert_format)
# en['links'] = en['links'].apply(convert_format)
# en['mentionedUsers'] = en['mentionedUsers'].apply(convert_format)

# es['hashtags'] = es['hashtags'].apply(convert_format)
# es['links'] = es['links'].apply(convert_format)
# es['mentionedUsers'] = es['mentionedUsers'].apply(convert_format)


In [30]:
en_combined = pd.concat([en, en_data], ignore_index=True)
es_combined = pd.concat([es, es_data], ignore_index=True)


In [2]:
# en_combined = pd.read_csv("en_combined.csv")
# es_combined = pd.read_csv("es_combined.csv")


In [3]:
# def convert_format(field):
#     return ast.literal_eval(field)

# # Convert string representation of lists into lists
# en_combined['hashtags'] = en_combined['hashtags'].apply(convert_format)
# en_combined['links'] = en_combined['links'].apply(convert_format)
# en_combined['mentionedUsers'] = en_combined['mentionedUsers'].apply(convert_format)

# es_combined['hashtags'] = es_combined['hashtags'].apply(convert_format)
# es_combined['links'] = es_combined['links'].apply(convert_format)
# es_combined['mentionedUsers'] = es_combined['mentionedUsers'].apply(convert_format)


In [4]:
en_combined.shape

(588839, 16)

In [35]:
# en_combined.to_csv("en_combined.csv",index=False)
# es_combined.to_csv("es_combined.csv",index=False)


### 2. Data exploration

#### 2.1 Popular hashtags

In [17]:
from collections import Counter

en_tags = []
es_tags = []

for hashtags in en_combined['hashtags']:
    if len(hashtags):
        en_tags.extend(tag['text'].lower() for tag in hashtags if 'text' in tag)
        
for hashtags in es_combined['hashtags']:
    if len(hashtags):
        es_tags.extend(tag['text'].lower() for tag in hashtags if 'text' in tag)
        
en_tag_freq = Counter(en_tags)
es_tag_freq = Counter(es_tags)

In [18]:
sum(en_tag_freq.values()) 

113855

In [19]:
sum(es_tag_freq.values()) 

168281

In [40]:
# total_en_tags = sum(en_tag_freq.values())  
# total_es_tags = sum(es_tag_freq.values())  

# en_tag_percent = Counter({tag: (count / total_en_tags) * 100 for tag, count in en_tag_freq.items()})
# es_tag_percent = Counter({tag: (count / total_es_tags) * 100 for tag, count in es_tag_freq.items()})

print("\nTop 10 English Tags by Frequency:")
for tag, frequency in en_tag_freq.most_common(10):
    print(f"{tag} {frequency}")

print("\nTop 10 Spanish Tags by Frequency:")
for tag, frequency in es_tag_freq.most_common(10):
    print(f"{tag} {frequency}")


Top 10 English Tags by Frequency:
trump2024 10499
maga 8441
trump 4553
biden 3382
bidenharris2024 1391
donaldtrump 1124
kamalaharris 934
gop 863
usa 734
joebiden 733

Top 10 Spanish Tags by Frequency:
biden 7163
trump2024 6260
trump 6194
eeuu 4457
maga 3211
mundo 2941
donaldtrump 2700
internacional 2509
estadosunidos 2252
usa 2159


#### 2.2 Popular domains

In [6]:
import tldextract

def extract_base_domain(url):
    extracted = tldextract.extract(url)
    return f"{extracted.domain}.{extracted.suffix}"


['dailymail.co.uk', 'cnn.com', 'example.co.uk']


In [21]:
from collections import Counter

en_links = []
es_links = []

for links in en_combined['links']:
    if len(links):
        en_links.extend(extract_base_domain(link['expanded_url']) for link in links if 'expanded_url' in link)
        
for links in es_combined['links']:
    if len(links):
        es_links.extend(extract_base_domain(link['expanded_url']) for link in links if 'expanded_url' in link)
        
en_link_freq = Counter(en_links)
es_link_freq = Counter(es_links)

In [13]:
total_en_links = sum(en_link_freq.values())  
total_es_links = sum(es_link_freq.values())  
print(f"Total domains shared in English data: {total_en_links}. Total domains shared in Spanish data: {total_es_links}")

Total domains shared in English data: 5847. Total domains shared in Spanish data: 6543


In [23]:
# total_en_links = sum(en_link_freq.values())  
# total_es_links = sum(es_link_freq.values())  

# en_link_percent = Counter({link: (count / total_en_links) * 100 for link, count in en_link_freq.items()})
# es_link_percent = Counter({link: (count / total_es_links) * 100 for link, count in es_link_freq.items()})

print("\nTop 10 English domains by Frequency:")
for link, freq in en_link_freq.most_common(10):
    print(f"{link} {freq}")

print("\nTop 10 Spanish domains by Frequency:")
for link, freq in es_link_freq.most_common(20):
    print(f"{link} {freq}")


Top 10 English domains by Frequency:
youtu.be 5155
x.com 3576
youtube.com 3003
dlvr.it 2258
foxnews.com 1887
breitbart.com 1407
trib.al 1341
msn.com 1126
smartnews.com 1118
newsbreak.com 1118

Top 10 Spanish domains by Frequency:
dlvr.it 7524
youtu.be 6001
youtube.com 3882
x.com 3355
bit.ly 3128
elpais.com 2349
buff.ly 2341
ift.tt 2223
ow.ly 1804
tinyurl.com 1755
short.gy 1572
trib.al 1354
acortar.link 1286
infobae.com 1220
rt.com 1088
clarin.com 1076
univision.com 1031
cnn.com 952
venezuela-news.com 799
uni.vi 781


In [44]:
print("Unqiue users for downsampled English data:",len(set(en_data['id_str']) ))

Unqiue users for downsampled English data: 146639


In [45]:
print("Unqiue users for downsampled Spanish data:",len(set(es_data['id_str'])))

Unqiue users for downsampled Spanish data: 83622


#### Check the amount of bilingual posters (case study)

In [49]:
users_en = set(en_combined['id_str'])  
users_es = set(es_combined['id_str'])

bilingual_posters = users_en.intersection(users_es)

bilingual_posters = list(bilingual_posters)



In [50]:
print(len(bilingual_posters), "users posted in English and Spanish")

13864 users posted in English and Spanish


In [25]:
# en_data.to_csv("en.csv", index=False)
# es_data.to_csv("es.csv", index=False)